In [3]:
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
plotly.offline.init_notebook_mode()

In [97]:
stations=pd.read_csv('station.csv')

In [2]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')
    
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')

In [3]:
import pyspark.sql as pysql

sqlCtx = pysql.SQLContext(sc)

trip_rdd=sc.textFile('trip.csv')
trips_df =pycsv.csvToDataFrame(sqlCtx,trip_rdd)

In [4]:
#el rdd de los trips
trips_data=trips_df.rdd

In [9]:
circular_trips=trips_data.filter(lambda x: x[4]==x[7])\
                        .map(lambda x:((x[3],x[6]),1))\
                        .reduceByKey(lambda x,y: x+y)\
                        .collect()

In [12]:
circular_trips=sorted(circular_trips,lambda x,y: y[1]-x[1])

## Cantidad de viajes circulares según estación

In [29]:

def get_coordenates(stations_list):
    """in : a list of ((station_name,station_name),int)
        out: a 2-column matrix with the lat and long of the stations
    """
    lat_lon=[[],[]]
    for trip,frec in circular_trips:
        lat=stations.loc[stations.name==trip[0],'lat']
        lon=stations.loc[stations.name==trip[0],'long']
        lat_lon[0].append(lat)
        lat_lon[1].append(lon)
    return lat_lon

In [81]:
import plotly.plotly as py
from plotly.graph_objs import *
mapbox_access_token = 'pk.eyJ1IjoibGVvY2VudHVyaW9uIiwiYSI6ImNqMGxsZnlyMTAwMDIzMm5vbXNidmtyOHEifQ.YqWBUbyhzglm38sJeRFl7A'

cities_coordenates=[((37.6158637,-122.192001),9,'San Francisco Bay'),((37.788925,-122.403452),12,'San Francisco'),
                    ((37.339301,-121.889937),12,'San Jose') ,((37.487682,-122.223492),12,'Redwood City'),
                    ((37.444521,-122.163093),12,'Palo Alto'),((37.389218,-122.081896),12,'Mountain View')]

lat_long=get_coordenates(circular_trips)
number_of_trips= ['{}'.format(frecuency) for trip,frecuency in circular_trips]
data = Data([
    Scattermapbox(
        lat = lat_long[0],
        lon = lat_long[1],
        text =['{} \n {}'.format(trip[0],frecuency) for trip,frecuency in circular_trips],
        mode='markers',
        marker = dict(
            
            color = number_of_trips,
            colorscale = [[0,"rgb(150,150,255)"],[1,"rgb(0, 0, 255)"]],
            opacity = 1,
            size = 9,
        )
    )
])
cities=[]
for coordenates,zoom,text in cities_coordenates:
    city=dict(
        args=[ { 
            'mapbox.center.lat':coordenates[0],
            'mapbox.center.lon':coordenates[1],
            'mapbox.zoom':zoom,
            'annotations[0].text':text
        } ],
        label=text,
        method='relayout'
    )
    cities.append(city)
updatemenus=list([
    dict(
        buttons = cities,
        x=1,
        y=1
    )
])
layout = Layout(
    autosize=False,
    width=700,
    height=800,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=37.6158637,
            lon=-122.192001

        ),
        pitch=0,
        zoom=9
        ),
    )
layout['updatemenus'] = updatemenus
fig = { 'data':data, 'layout':layout }
py.iplot(fig)

In [76]:
stations[stations.name=="Mountain View City Hall"]

,id,name,lat,long,dock_count,city,installation_date
20,27,Mountain View City Hall,37.389218,-122.081896,15,Mountain View,8/16/2013


In [5]:
trips=pd.read_csv('trip.csv')

In [17]:
trips_by_subscription=trips.loc[trips.start_station_id==trips.end_station_id,['subscription_type','id']]\
        .groupby('subscription_type').count().id.values

## Cantidad de viajes según tipo de subscripción

In [52]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = {
  "data": [
    {
      "values": [trips_by_subscription[0],trips_by_subscription[1]],
      "labels": [
        "Customers",
        "Subscribers",
      ],
      "hoverinfo":"label+percent",
      "hole": .4,
      "type": "pie"
    }
  ],
  "layout": {
        "title":"Cantidad de viajes según tipo de subscipción",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Cantidad de \n viajes",
                "x": 0.5,
                "y": 0.5
            }
        ]
    }
}
py.iplot(fig)

* Hay mayor cantidad de 'customers', lo que concuerda con nuestra suposición de que los viajes circulares son realizados por turistas.

## Cantidad de viajes según día de la semana

In [36]:
import datetime
trips.start_date=trips.start_date.map(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y %H:%M') )

In [57]:
trips['day_of_week']=trips.start_date.map(lambda x: x.dayofweek)
trips_by_day_of_week=trips.loc[trips.start_station_id==trips.end_station_id,['subscription_type','id','day_of_week']]\
                    .groupby('day_of_week').count().id.values
trips_by_day_of_week_subs=trips.loc[(trips.start_station_id==trips.end_station_id) & \
                        (trips.subscription_type=='Subscriber'),['subscription_type','id','day_of_week']]\
                        .groupby('day_of_week').count().id.values
trips_by_day_of_week_cust=trips.loc[(trips.start_station_id==trips.end_station_id) & \
                        (trips.subscription_type=='Customer'),['subscription_type','id','day_of_week']]\
                        .groupby('day_of_week').count().id.values

In [77]:
def plot_bars(labels,values,names):
    data=[]
    for i in range(len(labels)):
        data.append(
                  go.Bar(
                    x=labels[i],
                    y=values[i],
                    name=names[i]
            )
        )
    trace1=go.Bar(
                x=labels[0],
                y=trips_by_day_of_week,
                name='Total'
            )
    trace2=go.Bar(
                x=['Lunes', 'Martes', 'Miercoles','Jueves','Viernes','Sabado','Domingo'],
                y=trips_by_day_of_week_subs,
                name='Subscribers'
            )
    trace3=go.Bar(
                x=['Lunes', 'Martes', 'Miercoles','Jueves','Viernes','Sabado','Domingo'],
                y=trips_by_day_of_week_cust,
                name='Customers'
            )

    data = [trace1,trace3,trace2]

    iplot(data)


In [78]:
dias=['Lunes', 'Martes', 'Miercoles','Jueves','Viernes','Sabado','Domingo']
labels=[dias,dias,dias]
values=[trips_by_day_of_week,trips_by_day_of_week_cust,trips_by_day_of_week_subs]
names=['Total','Customers','Subscribers']
plot_bars(labels,values,names)

* La cantidad de viajes los fines de semana aumenta considerablemente. Además la diferencia entre subscribers y customers aumenta notablemente.

## Según mes

In [70]:
trips['start_month']=trips.start_date.map(lambda x: x.month)

In [72]:
trips_by_month=trips.loc[trips.start_station_id==trips.end_station_id,['subscription_type','id','start_month']]\
                    .groupby('start_month').count().id.values
trips_by_month_subs=trips.loc[(trips.start_station_id==trips.end_station_id) & \
                        (trips.subscription_type=='Subscriber'),['subscription_type','id','start_month']]\
                        .groupby('start_month').count().id.values
trips_by_month_cust=trips.loc[(trips.start_station_id==trips.end_station_id) & \
                        (trips.subscription_type=='Customer'),['subscription_type','id','start_month']]\
                        .groupby('start_month').count().id.values

In [80]:
meses=['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
labels=[meses,meses,meses]
values=[trips_by_month,trips_by_month_cust,trips_by_month_subs]
names=['Total','Customers','Subscribers']
plot_bars(labels,values,names)

## Por hora

In [82]:
trips['start_hour']=trips.start_date.map(lambda x: x.hour)

In [83]:
trips_by_hour=trips.loc[trips.start_station_id==trips.end_station_id,['subscription_type','id','start_hour']]\
                    .groupby('start_hour').count().id.values
trips_by_hour_subs=trips.loc[(trips.start_station_id==trips.end_station_id) & \
                        (trips.subscription_type=='Subscriber'),['subscription_type','id','start_hour']]\
                        .groupby('start_hour').count().id.values
trips_by_hour_cust=trips.loc[(trips.start_station_id==trips.end_station_id) & \
                        (trips.subscription_type=='Customer'),['subscription_type','id','start_hour']]\
                        .groupby('start_hour').count().id.values

In [84]:
horas=range(24)
labels=[horas,horas,horas]
values=[trips_by_hour,trips_by_hour_cust,trips_by_hour_subs]
names=['Total','Customers','Subscribers']
plot_bars(labels,values,names)

* ¿Parada para almorzar?

## Según ciudad

In [102]:
trips['city']=trips.start_station_id.map(lambda x: stations.loc[stations.id==x,'city'].values[0] )

In [111]:
# trips_by_hour=trips.loc[trips.start_station_id==trips.end_station_id,['subscription_type','id','start_hour']]\
#                     .groupby('start_hour').count().id.values
# trips_by_hour_subs=trips.loc[(trips.start_station_id==trips.end_station_id) & \
#                         (trips.subscription_type=='Subscriber'),['subscription_type','id','start_hour']]\
#                         .groupby('start_hour').count().id.values
# trips_by_hour_cust=trips.loc[(trips.start_station_id==trips.end_station_id) & \
#                         (trips.subscription_type=='Customer'),['subscription_type','id','start_hour']]\
#                         .groupby('start_hour').count().id.values
cities=['San Francisco','San Jose','Palo Alto','Redwood City','Mountain View']
trips_by_city=[]
trips_by_city_subs=[]
trips_by_city_cust=[]
for city in cities:
    condition=(trips.start_station_id==trips.end_station_id)&(trips.city==city)
    trips_by_city.append( trips.loc[condition,['id','city'] ].groupby('city').count().id.values[0])
                         
    condition=(trips.start_station_id==trips.end_station_id)&\
                    (trips.city==city)&(trips.subscription_type=='Subscriber')
    trips_by_city_subs.append( trips.loc[condition,['subscription_type','id','city']]\
                        .groupby('city').count().id.values[0] )
                         
    condition=(trips.start_station_id==trips.end_station_id ) &\
                    (trips.city==city)&(trips.subscription_type=='Customer')
    trips_by_city_cust.append( trips.loc[condition,['subscription_type','id','city']]\
                    .groupby('city').count().id.values[0] )

In [113]:
labels=[cities,cities,cities]
values=[trips_by_city,trips_by_city_cust,trips_by_city_subs]
names=['Total','Customers','Subscribers']
plot_bars(labels,values,names)